In [1]:
%pip install -r requirements.txt

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached charset_normalizer-3.4.0-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyYAML-6.0.2-cp

In [3]:
%pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121Note: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 8.4 MB/s eta 0:04:51
     ---------------------------------------- 0.0/2.4 GB 10.8 MB/s eta 0:03:48
     ---------------------------------------- 0.0/2.4 GB 11.2 MB/s eta 0:03:39
     ---------------------------------------- 0.0/2.4 GB 11.4 MB/s eta 0:03:35
     ---------------------------------------- 0.0/2.4 GB 11.4 MB/s eta 0:03:34
     ---------------------------------------- 0.0/2.4 GB 11.5 MB/s eta 0:03:32
     ---------------------------------------- 0.0/2.4 GB 11.5 MB/s eta 0:03:31
     ---------------------------------------- 0.0/2.4 GB 11.6 MB/s eta 0:03:30
     ---------------------------------------- 0.0/2.4 GB 11.7 MB/s eta 0:03:29
     ---------------------------------------- 0.0/2.4 GB 11.6 MB/s eta 0:03:29
     ----------

In [ ]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


[Info](https://whimsical.com/simple-local-rag-workflow-39kToR3yNf7E8kY4sS2tjV)

In [5]:
import os
import requests

dataDir = "./data/human-nutrition-text.pdf"

if not os.path.exists(dataDir):
    print(f"Doesn't exist {dataDir}")
else:
    print(f"File exists {dataDir}")


File exists ./data/human-nutrition-text.pdf


In [ ]:
import fitz 
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    return text.replace('\n', ' ').strip()


def read_pdf(path: str) -> list[dict]:
    doc = fitz.open(path)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_text.append({"page_number":page_number-41,
                               "page_char_count":len(text),
                               "page_word_count":len(text.split(" ")),
                               "page_sentence_count":len(text.split(". ")),
                               "page_token_count":len(text / 4),
                               "page_text":text})

In [ ]:
pages_and_text = read_pdf(dataDir)

pages_and_text[:2]

0it [00:00, ?it/s]

TypeError: unsupported operand type(s) for /: 'str' and 'int'